<a href="https://colab.research.google.com/github/BaishaliChetia/CapsNet-Keras/blob/master/b_mnistTf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Original implementation at:

https://github.com/ageron/handson-ml/blob/master/extra_capsnets.ipynb

Geron's model doesn't use the keras functional API. In the keras functional API, you don't need to give the batchsize. 

When you print the model, you get this:

```
Layer (type)                 Output Shape              Param #   
_________________________________________________________________
input (InputLayer)           [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv_layer_1 (Conv2D)        (None, 20, 20, 256)       20992     
_________________________________________________________________
conv_layer_2 (Conv2D)        (None, 6, 6, 256)         5308672   
_________________________________________________________________
reshape_layer_1 (Reshape)    (None, 1, 1152, 8)        0         
_________________________________________________________________
caps1_output_layer (SquashLa (None, 1, 1152, 8)        0         
_________________________________________________________________
Total params: 5,329,664
Trainable params: 5,329,664
Non-trainable params: 0
```

Notice that the Input-layer has shape (None, 28, 28, 1), but we only specified (28, 28, 1). It added None implicitly and that takes care of the batch.

So for anywhere Geron uses the batch size explicitly, you don't need to do anything and tensorflow will take care of.

Also note that tensorflow 1 APIs are still provided with the compat layer. I used the reduce_sum from TF1 in the squash layer, that allowed me to use Geron's code.

Documentation on how to migrate from TF1 to TF2 can be found here:

https://www.tensorflow.org/guide/migrate


In [4]:
import numpy as np
import tensorflow as tf
import pandas as pd
import tensorflow.keras as K

In [5]:
caps1_n_maps = 32
caps1_n_caps = caps1_n_maps * 6 * 6  # 1152 primary capsules
caps1_n_dims = 8
caps2_n_caps = 10
caps2_n_dims = 16


In [6]:
class SquashLayer(K.layers.Layer):
  def __init__(self, axis=-1, **kwargs):
    super(SquashLayer, self).__init__(**kwargs)
    self.axis = axis
    
  def build(self, input_shapes):
    #print("SquashLayer input_shape=", input_shapes)
    pass

  def call(self, inputs):
    EPSILON = 1.0e-9
    squared_norm = tf.compat.v1.reduce_sum(tf.square(inputs),\
                                           axis=self.axis,\
                                           keepdims=True)
    safe_norm = tf.sqrt(squared_norm + EPSILON)
    squash_factor = squared_norm / (1. + squared_norm)
    unit_vector = inputs / safe_norm
    return squash_factor * unit_vector


In [25]:
class SafeNorm(K.layers.Layer):
  def __init__(self, axis=-1,  **kwargs):
    super(SafeNorm, self).__init__(**kwargs)
    self.axis = axis
  def build(self, input_shapes):
    pass
  def call(self, input, keep_dims = False):
    EPSILON = 1.0e-9
    squared_norm = tf.compat.v1.reduce_sum(tf.square(inputs), axis=self.axis, keepdims= keep_dims)
    safe_norm = tf.sqrt(squared_norm + EPSILON)
    return safe_norm



In [28]:
# This should be the part where the digit layer, and where we tile things
# This is incomplete, and work in progress
# TODO: Complete this
class MyDigitCapsLayer(K.layers.Layer):
  def __init__(self, **kwargs):
    super(MyDigitCapsLayer, self).__init__(**kwargs)

  def build(self, input_shapes):
    init_sigma = 0.1
    #                                    print("input_shapes = ", input_shapes)
    self.kernel = self.add_weight(\
                      "kernel", 
                      (caps1_n_caps, caps2_n_caps, caps2_n_dims, caps1_n_dims),
                      initializer="random_normal", dtype=tf.float32)
    #print("My Caps Layer2: self.kernel = ", self.kernel.shape, "input_shape = ", input_shapes)

  
  # To debug this function, I used prints to print the shape
  # expand_dims just adds an exis, so if you say expand_dims(inshape=(5, 3), -1),
  # you get the output shape (5, 3, 1), it just adds an axis at the end
  # Then tile just multiplies one of the dimensions (that is it stacks along that direction N times)
  # so tile(inshape=(5, 3, 1), [1, 1, 1000]) will yield a shape (5, 3, 1000)
  #
  # Notice I didn't tile in build, but in call, Most probaly this is the right thing to do
  # but we'll only figure out when we actually train
  def call(self, inputs):
    #                                    print("==>", inputs.shape)
    # Add a dimension at the end
    exp1 = tf.expand_dims(inputs, -1, name="caps1_output_expanded")
    #                                    print("exp1 ==>", exp1.shape)
    # add a dimension along 3rd axis
    exp1 = tf.expand_dims(exp1, 2, name="caps2_output_espanced")
    #                                    print("exp1 ==>", exp1.shape)
    # tile along 3rd axis
    tile = tf.tile(exp1, [1, 1, caps2_n_caps, 1, 1], name="caps1_output_tiled")
    #                                    print("tile ==> ", tile.shape)
    caps2_predicted = tf.matmul(self.kernel, tile, name="caps2_predicted")
    #                                    print("multiply ==>", caps2_predicted.shape)
    return caps2_predicted

In [9]:
class RoutingByAgreement(K.layers.Layer):
  def __init__(self, roundno=-1, agreement=None, **kwargs):
    super(RoutingByAgreement, self).__init__(**kwargs)
    self.round_number = roundno
    self.agreement = agreement

  def build(self, input_shapes):
    self.raw_weights_1 = self.add_weight("raw_weights", \
                                         (caps1_n_caps, caps2_n_caps, 1, 1), \
                                         initializer = "zeros", \
                                         dtype=tf.float32,)
    
    #print("Routing layer: self.raw_weights = ", self.raw_weights.shape, "input_shape = ", input_shapes)


  def squash(self, inputs):
    EPSILON = 1.0e-9
    squared_norm = tf.compat.v1.reduce_sum(tf.square(inputs),\
                                           axis=self.axis,\
                                           keepdims=True)
    safe_norm = tf.sqrt(squared_norm + EPSILON)
    squash_factor = squared_norm / (1. + squared_norm)
    unit_vector = inputs / safe_norm
    return squash_factor * unit_vector
    
  def single_round_routing(self, inputs):
    pass

  def call(self, inputs):
    if self.round_number == 1:
      self.raw_weights = self.raw_weights_1
    
    elif self.round_number == 2:
      self.raw_weights = tf.add(self.raw_weights_1,\
                                agreement,\
                                name="raw_weights_round_2")
    
    value = inputs
    for i in range(num_routes):
      value = single_round_routing(self, value)

    self.routing_weights = tf.nn.softmax(self.raw_weights,\
                                         axis=2,\
                                         name="routing_weights")
    
    weighted_predictions = tf.multiply(self.routing_weights, inputs,
                                   name="weighted_predictions")
    weighted_sum = tf.reduce_sum(weighted_predictions, axis=1, keepdims=True,
                             name="weighted_sum")
    
    return weighted_sum


In [1]:
# https://www.tensorflow.org/api_docs/python/tf/keras/losses/Loss
class MarginLoss(K.losses.Loss):
    def __init__(self, **kwargs):
      super(MarginLoss, self).__init__(**kwargs)
    
    def call(self, y_true, y_pred):
      print(y_true, y_true.shape)
      print(y_pred, y_pred.shape)
      return 0


def margin_loss(input): #input = caps2_output_round_2
  y = K.Input(shape=[], dtype=tf.int64, name="y")
  m_plus = 0.9
  m_minus = 0.1
  lambda_ = 0.5 
  T = tf.one_hot(y, depth=caps2_n_caps, name="T")
  print("T:", T)
  caps2_output_norm = SafeNorm()(input, keep_dims = True)
  
  present_error_raw = tf.square(tf.maximum(0., m_plus - caps2_output_norm),
                              name="present_error_raw")
  present_error = tf.reshape(present_error_raw, shape=(-1, 10),
                           name="present_error")  
  
  absent_error_raw = tf.square(tf.maximum(0., caps2_output_norm - m_minus),
                             name="absent_error_raw")
  absent_error = tf.reshape(absent_error_raw, shape=(-1, 10),
                          name="absent_error")
  
  L = tf.add(T * present_error, lambda_ * (1.0 - T) * absent_error,
           name="L")
  marginLoss = tf.reduce_mean(tf.reduce_sum(L, axis=1), name="margin_loss")
  return marginLoss

NameError: ignored

In [21]:


class Model:
    @staticmethod
    def build(inshape=(28, 28, 1)):
        inp = K.Input(shape=inshape, dtype=tf.float32, name='input')
        
        # Primary capsules
        # For each digit in the batch
        # 32 maps, each 6x6 grid of 8 dimensional vectors
        
        # First Conv layer
        conv1_params = \
        {
            "filters": 256,
            "kernel_size": 9,
            "strides": 1,
            "padding": "valid",
            "activation": tf.nn.relu,
        }
        x = K.layers.Conv2D(**conv1_params, name="conv_layer_1")(inp)
        
        # Second conv layer
        conv2_params = \
        {
            "filters": caps1_n_maps * caps1_n_dims, # 256 convolutional filters
            "kernel_size": 9,
            "strides": 2,
            "padding": "valid",
            "activation": tf.nn.relu
        }
        x = K.layers.Conv2D(**conv2_params, name="conv_layer_2")(x)
        
        # Reshape
        x = K.layers.Reshape((caps1_n_caps, caps1_n_dims), name="reshape_layer_1")(x)
        x = SquashLayer(name="caps1_output_layer")(x)
        
        # The next part is to build the digit caps, starts from
        # 4:38 in the video
        # I haven't got to it yet, this will take time
        # at any point of time print the shape, and it will print it
        x = MyDigitCapsLayer(name = "caps2_predicted")(x)
        caps2_predicted = x # Save this value for later
        
        #routing by agreement
        #Round 1
        x = RoutingByAgreement(name="routing1", roundno=1, agreement=None)(x)
        x = SquashLayer(name= "caps2_output_round_1", axis= -2)(x)


        #Round2
        #tiling
        x = tf.tile(\
                    x ,\
                    [1, caps1_n_caps, 1, 1, 1],\
                    name="caps2_output_round_1_tiled")
        #agreement
        agreement = tf.matmul(\
                              caps2_predicted, \
                              x,\
                              transpose_a=True,\
                              name="agreement")
        print(f"x.shape = {x.shape}, caps2_predicted.shape = {caps2_predicted.shape}, agreement.shape = {agreement.shape}")
        
        x = RoutingByAgreement(name="routing2", roundno=2, agreement=agreement)(x)
        x = SquashLayer(name= "caps2_output_round_2", axis= -2)(x)
        
        return K.Model(inputs=inp, outputs=x, name='my')
    
m = Model.build()

# Print the final shape
print(m, type(m), m.summary())




My Caps Layer2: self.kernel =  (1152, 10, 16, 8) input_shape =  (None, 1152, 8)
Model: "my"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv_layer_1 (Conv2D)           (None, 20, 20, 256)  20992       input[0][0]                      
__________________________________________________________________________________________________
conv_layer_2 (Conv2D)           (None, 6, 6, 256)    5308672     conv_layer_1[0][0]               
__________________________________________________________________________________________________
reshape_layer_1 (Reshape)       (None, 1152, 8)      0           conv_layer_2[0][0]               
_________________